In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir("/content/gdrive/My Drive/Saki_2019/data/flair")

In [7]:
# download flair library #
! pip install flair

     |████████████████████████████████| 143kB 6.4MB/s 
     |████████████████████████████████| 798kB 40.4MB/s 
     |████████████████████████████████| 655kB 40.8MB/s 
     |████████████████████████████████| 133kB 35.7MB/s 
     |████████████████████████████████| 1.0MB 41.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
Successfully built mpld3 regex segtok sqlitedict


In [9]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher

## make sure this describes your file structure
columns = {0: 'text', 1: 'ner'}

# folder where training and test data are
data_folder = '/content/gdrive/My Drive/Saki_2019/data/flair'

# 1.0 is full data, try a much smaller number like 0.1 to test run the code
downsample = 1 

## your train file name
train_file = 'train_res_bilou.txt'

## your test file name
test_file = 'test_res_bilou.txt'
# 1. get the corpus
corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns, train_file=train_file, test_file=test_file, dev_file=None).downsample(downsample)
print(corpus)

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')
print(tag_dictionary.idx2item)

2019-06-18 13:47:51,286 Reading data from /content/gdrive/My Drive/Saki_2019/data/flair
2019-06-18 13:47:51,287 Train: /content/gdrive/My Drive/Saki_2019/data/flair/train_res_bilou.txt
2019-06-18 13:47:51,292 Dev: None
2019-06-18 13:47:51,293 Test: /content/gdrive/My Drive/Saki_2019/data/flair/test_res_bilou.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


Corpus: 315 train + 35 dev + 87 test sentences
[b'<unk>', b'O', b'O"', b'Companies', b'-"', b'Degree"', b'Skills"', b'<START>', b'<STOP>']


In [11]:
from typing import List

# 4. initialize embeddings. Experiment with different embedding types to see what gets the best results
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings,FlairEmbeddings

embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings (needs a LONG time to train :-)
    #FlairEmbeddings('news-forward'),
    #FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type='ner',
                                        use_crf=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [12]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

## give your model a name and folder of your choice. Your model will be saved there for loading later 
## you can run this notebook many times with different embeddings/params and save the models with different names
model_name = 'resources/taggers/resume-ner-4'

# 7. start training - you can experiment with batch size if you get memory errors
# how many epochs does it take before the model stops showing improvement? Start with a big number like 150, and stop the code cell
# from running at any time - the framework will persist the best model even if you interrupt training. 
trainer.train(model_name,
              learning_rate=0.1,
              mini_batch_size=32,
              #anneal_with_restarts=True,
              max_epochs=150)

2019-06-18 13:51:15,715 ----------------------------------------------------------------------------------------------------
2019-06-18 13:51:15,722 Evaluation method: MICRO_F1_SCORE
2019-06-18 13:51:16,042 ----------------------------------------------------------------------------------------------------
2019-06-18 13:51:21,393 epoch 1 - iter 0/10 - loss 1982.46289062
2019-06-18 13:51:29,960 epoch 1 - iter 1/10 - loss 2067.35180664
2019-06-18 13:51:36,583 epoch 1 - iter 2/10 - loss 1802.03527832
2019-06-18 13:51:40,370 epoch 1 - iter 3/10 - loss 1529.00665283
2019-06-18 13:51:46,864 epoch 1 - iter 4/10 - loss 1346.66887207
2019-06-18 13:51:52,464 epoch 1 - iter 5/10 - loss 1168.29695638
2019-06-18 13:51:56,175 epoch 1 - iter 6/10 - loss 1028.02293614
2019-06-18 13:52:00,096 epoch 1 - iter 7/10 - loss 929.84697723
2019-06-18 13:52:04,829 epoch 1 - iter 8/10 - loss 851.04266018
2019-06-18 13:52:08,346 epoch 1 - iter 9/10 - loss 782.64784088
2019-06-18 13:52:08,854 ---------------------

{'dev_loss_history': [tensor(219.5380, device='cuda:0'),
  tensor(174.8539, device='cuda:0'),
  tensor(154.1482, device='cuda:0'),
  tensor(142.6956, device='cuda:0'),
  tensor(135.3738, device='cuda:0'),
  tensor(119.5223, device='cuda:0'),
  tensor(115.7494, device='cuda:0'),
  tensor(112.4148, device='cuda:0'),
  tensor(111.3802, device='cuda:0'),
  tensor(108.5417, device='cuda:0'),
  tensor(107.6384, device='cuda:0'),
  tensor(106.3457, device='cuda:0'),
  tensor(106.2371, device='cuda:0'),
  tensor(104.3350, device='cuda:0'),
  tensor(103.4868, device='cuda:0'),
  tensor(103.7613, device='cuda:0'),
  tensor(103.6114, device='cuda:0'),
  tensor(102.4051, device='cuda:0'),
  tensor(102.4545, device='cuda:0'),
  tensor(101.9461, device='cuda:0'),
  tensor(101.8941, device='cuda:0'),
  tensor(101.5799, device='cuda:0'),
  tensor(101.5165, device='cuda:0'),
  tensor(101.4032, device='cuda:0'),
  tensor(101.3834, device='cuda:0'),
  tensor(101.2200, device='cuda:0'),
  tensor(101.1980,